In [1]:
def preprocessor(df1,date):
    import pandas as pd
    import numpy as np
    from sklearn import preprocessing
    #df1=df1[df1['Status']=='Overdue']
    #print(df1['Status'])
    df1.dropna(inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    
    
    
    df1['Pstng.Date']=pd.to_datetime(df1['Pstng.Date'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    df1['Net.Due.Dt']=pd.to_datetime(df1['Net.Due.Dt'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    df1['Clearing']=pd.to_datetime(df1['Clearing']).dt.strftime("%Y%m%d").astype(int)   
    df=df1[df1['Pstng.Date'] < date ]
    df3=df[df['Clearing'] > date] 
    df.drop(df[df['Clearing'] > date].index,inplace=True)
    df1=df
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    
    
    df3 = df3.reset_index()
    df3.drop(columns='index',inplace=True)
    
    
    df2=df1[['Pstng.Date','Net.Due.Dt','Clearing','Status']]
    df2.dropna(inplace=True)
    df2['Range of Delay']=pd.to_datetime(df2['Clearing'],format='%Y%m%d') - pd.to_datetime(df2['Net.Due.Dt'],format='%Y%m%d')
    df2['PayT']= pd.to_datetime(df2['Net.Due.Dt'],format='%Y%m%d') - pd.to_datetime(df2['Pstng.Date'],format='%Y%m%d')
    df2['Range of Delay']=df2['Range of Delay'].dt.days
    df2['Range of Delay']=pd.to_numeric(df2['Range of Delay'])
    df2['PayT']=df2['PayT'].dt.days
    df2['PayT']=pd.to_numeric(df2['PayT'])
    
    df2 = df2.reset_index()
    df2.drop(columns = 'index',inplace=True)
    
    df2['Status'] = np.where(df2['Range of Delay'] > 7, 2, (np.where(df2['Range of Delay'] > 0, 1,0)))
    print(np.unique(df2['Status'],return_counts=True))
    print('----------------------------------------------------------------------------------------')
    
    df1['Status']=df2['Status']
    df1['PayT']=df2['PayT']
    df1.dropna(inplace=True)
    y=df1['Status']
    
    
    busa = preprocessing.LabelEncoder()
    ccar= preprocessing.LabelEncoder()
    month=preprocessing.LabelEncoder()
    zone=preprocessing.LabelEncoder()
    bran=preprocessing.LabelEncoder()
    payt=preprocessing.LabelEncoder()
    
    df1.drop(columns='Status',inplace=True)
    #df1['Status']=df1['Status'].astype('category').cat.codes
    arr=busa.fit(df1['BusA'])
    df1['BusA']=busa.transform(df1['BusA'])
    arr=ccar.fit(df1['CCAr'])
    df1['CCAr']=ccar.transform(df1['CCAr'])#.astype('category').cat.codes
    #df1['Account']=df1['Account'].astype('category').cat.codes
    arr=month.fit(df1['Month'])
    df1['Month']=month.transform(df1['Month'])#.astype('category').cat.codes
    df1.drop(columns='Reference',inplace=True)
    df1.drop(columns='Customer.Name',inplace=True)
    df1.drop(columns=['DocumentNo','Year','Clrng.doc.'],inplace=True)
    arr=zone.fit(df1['Zone'])
    df1['Zone']=zone.transform(df1['Zone'])    #.astype('category').cat.codes
    arr=bran.fit(df1['Bran'])
    df1['Bran']=bran.transform(df1['Bran'])#.astype('category').cat.codes
    arr =payt.fit(df1['PayT'])
    #df1['PayT'] = payt.transform(df1['PayT'])#.astype('category').cat.codes
    df1.drop(columns='Doc.Chq.dt',inplace=True)
    df1.drop(columns='Ty',inplace=True)
    df1.drop(columns='Sale.Type',inplace=True)
    
    
    #df1.drop(columns='Clearing',inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    df1.drop(columns='Arr..Clearing...Net.Due.Date.',inplace=True)
    df1.drop(columns='G.L',inplace=True)
    df1.drop(columns='Clearing',inplace=True)
    

    
    
    #print('Creating Red Alert Zone')
    #table = pd.crosstab(columns=y, index=df1['Account'])
    #table.columns= ["No delay"," 0 - 10", ">10"]
    #print(table)
    #print(table.sort_values(['>10'],ascending=False).head())
    #arr=table.sort_values(['>10'],ascending=False)
    #arr.drop(columns=["No delay"," 0 - 10"],inplace=True)
    #df_1=arr[arr['>10']>40]
    #df_2=[]
    
    
    #df1['Account']=df1['Account']/100000
    #df1['Reference']=df1['Reference']/1000000000
    #df1['Pstng.Date']=df1['Pstng.Date']/100000000
    #df1['Net.Due.Dt']=df1['Pstng.Date']/100000000
    


    #print(df_1.index)
    
    
    ###################Oversampling##############################
    #from imblearn.over_sampling import ADASYN 
    #sm = ADASYN()
    #df4, y = sm.fit_resample(df1, y)
    #df1=pd.DataFrame(df4,columns=df1.columns)
    ############################################################
    
    #y1=df1.columns
    #x = df1.values #returns a numpy array
    #min_max_scaler = preprocessing.MinMaxScaler()
    #x_scaled = min_max_scaler.fit_transform(x)
    #df1 = pd.DataFrame(x_scaled,columns=df1.columns)
    
    print('Y s in the set')
    print(np.unique(y,return_counts=True))
    print('----------------------------------------------------------------------------------------')
    
    return df1,df3,y,busa,ccar,month,zone,bran,payt

In [ ]:
def trainer(df1,y):
    import pandas as pd
    import numpy as np
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    import keras
    from keras.models import Sequential
    from keras.layers import Dense 
    #y=np.array(df1['Status'])
    
    

    X=df1
    #df1.head(5)
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3, random_state=0)
    
    
    y_train = keras.utils.to_categorical(ytrain, 3)
    y_test = keras.utils.to_categorical(ytest, 3)
    
    model = Sequential()
    model.add(Dense(10, input_dim=len(xtrain.columns), activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(6,  activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(3, activation='softmax'))

# compile and fit model
    model.compile(loss="categorical_crossentropy", optimizer="sgd")
    model.fit(xtrain, y_train, batch_size=15, epochs=20, learning_rate=0.0000001 ,validation_data=(xtest, y_test))
    
    eclf1=model.predict(xtrain)
    eclf1=np.argmax(eclf1,axis=1)
    eclf2=model.predict(xtest)
    eclf2=np.argmax(eclf2,axis=1)
    print('TRAINING RESULTS-----------------------------------------')
    print("Accuracy: for Training :",metrics.accuracy_score(ytrain, eclf1)*100)
    print("Accuracy: for Testing :",metrics.accuracy_score(ytest, eclf2)*100)
    print()
    print('Confusion Matrix for Random Forest')
    print(metrics.confusion_matrix(eclf2, ytest))
    print()
    print('--------------------------------------------------------------------------------------------------------------------')
    return model

In [3]:
def predict(df1,clf,busa,ccar,month,zone,bran,payt):
    import numpy as np
    import pandas as pd
    import copy
    from sklearn import preprocessing
    df=copy.deepcopy(df1)
    print(df.shape)
    df['Prediction']=np.zeros(len(df1))
    print(df1.shape)
    df1.dropna(inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    #df1['Status']=df1['Status'].astype('category').cat.codes)
    #df1['Status']=df1['Status'].astype('category').cat.codes
    
    df1['PayT']= pd.to_datetime(df1['Net.Due.Dt'],format='%Y%m%d') - pd.to_datetime(df1['Pstng.Date'],format='%Y%m%d')
    df1['PayT']=df1['PayT'].dt.days
    df1['PayT']=pd.to_numeric(df1['PayT'])
    df1.drop(columns='Clearing',inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns = 'index',inplace=True)
    #df1['Status']=df1['Status'].astype('category').cat.codes
    df1['BusA']=busa.transform(df1['BusA'])
    df1['CCAr']=ccar.transform(df1['CCAr'])#.astype('category').cat.codes
    #df1['Account']=df1['Account'].astype('category').cat.codes
    df1['Month']=month.transform(df1['Month'])#.astype('category').cat.codes
    df1.drop(columns='Reference',inplace=True)
    df1.drop(columns='Customer.Name',inplace=True)
    df1.drop(columns=['DocumentNo','Year'],inplace=True)
    df1['Zone']=zone.transform(df1['Zone'])#.astype('category').cat.codes
    df1['Bran']=bran.transform(df1['Bran'])#.astype('category').cat.codes
    #df1['PayT'] =payt.transform(df1['PayT'])#.astype('category').cat.codes
    df1.drop(columns='Doc.Chq.dt',inplace=True)
    df1.drop(columns='Ty',inplace=True)
    df1.drop(columns='Sale.Type',inplace=True)
    #df1['Pstng.Date']=pd.to_datetime(df1['Pstng.Date'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    #df1['Net.Due.Dt']=pd.to_datetime(df1['Net.Due.Dt'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    #df1['Clearing']=pd.to_datetime(df1['Clearing']).dt.strftime("%Y%m%d").astype(str)
    #df1.drop(columns=['Net.Due.Dt','Clearing'],inplace=True)
    #df1.drop(columns='Net.Due.Dt',inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    df1.drop(columns='Arr..Clearing...Net.Due.Date.',inplace=True)
    df1.drop(columns='G.L',inplace=True)
    
    #y1=df1.columns
    #x = df1.values #returns a numpy array
    #min_max_scaler = preprocessing.MinMaxScaler()
    #x_scaled = min_max_scaler.fit_transform(x)
    #df1 = pd.DataFrame(x_scaled,columns=df1.columns)
    
    #df1['Account']=df1['Account']/100000
    #df1['Reference']=df1['Reference']/1000000000
    #df1['Pstng.Date']=df1['Pstng.Date']/100000000
    #df1['Net.Due.Dt']=df1['Pstng.Date']/100000000
    
    
    print('Starting Prediction ....')
    prediction=clf.predict(df1)
    print('Generating Probabilities.....')
    predictprob=clf.predict_proba(df1)
    print('Finished prediction...')
    print()
    #df['Probability']=np.zeros(len(df))
    print(df1.head())
    

    arr=['No Delay','Low Delay','High Delay']
    arr1=['Low delay','Low Medium Delay','High Delay']#,'High Delay']#,'30 to 180','>180']#,'8-15 days','16-30 days','31-60 days','61-90 days','90-180 days','>180 days']
    for i in range(len(arr)):
        df[arr1[i]]=np.empty(len(df))
    print('Computing Prediction Table............')
    #for i in range(len(prediction)):
        #flag=0
        #if ( i % 500 == 0):
         #   print('----------------------------------------------------------------------------------------------------------- i =',i)
        #for k in range(len(redzone)):
           # if df['Account'][i] == redzone[k] :
             #   df['Prediction'][i]='>=10'
              #  flag=1
              #  print('----------------------------------------------------------------RED ALERT------------------------------------------ ')
                
        #if(flag == 0):
    
        #print(arr[prediction[i]])
        #arr2=""
        #print('Actual:',df['Arr..Clearing...Net.Due.Date.'][i])
        #print('--------------------------------------------------------')
    df['Prediction']=prediction
    #df['Prediction']=np.where(df['Prediction'] == 0, 'No Delay',(np.where(df['Prediction'] == 1, 'Low Delay',(np.where(df['Prediction'] == 2, 'High Delay','')))))
    for i in range(len(arr1)):
        df[arr1[i]]=np.around(predictprob[:,i],decimals=2)
        
    print(df.head(5))
    #df['Payment Within Next Few Days']=np.where(df['Prediction'] == 1, 'Yes',(np.where((df['Low delay']>0.2) & (df['High Delay']<0.4)),'Yes','No'))
    print()
    print(df.head())
    print('Writing to csv file')
    print('-----------------------------------------------------------------------------------------')
    df.to_csv('Predictedout6.csv',index=False)

In [5]:
# import time
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import copy
trainpath='MyData.csv'

print('Reading data...')
df1=pd.read_csv(trainpath)
print('Finished Reading...')
print()
print(df1.head(5))
print()
date=20181220
df3=copy.deepcopy(df1)
print('Preprocessing started .......')
df2,df4,y,busa,ccar,month,zone,bran,payt=preprocessor(df1,date)
print(df2.head(5))
print('Preprocessing Done...........')
print('--------------------------------------------------------------------------------------------------------')
#print('RED ZONE VISUALISATION....')

#visualisation_red(df3)

Reading data...
Finished Reading...

   BusA CCAr     Reference  Account          Customer.Name   Zone  Bran  Ty  \
0  1000  BPR  4.132000e+09   929488      KAUSALYA AGENCIES  South  SBAD  RV   
1  9600  BPR  1.115502e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   
2  9600  BPR  1.115502e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   
3  1000  BPR  2.412166e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   
4  2500  BPR  2.410320e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   

    Status  Local.Crcy.Amt  ...  Doc.Chq.dt Month Pstng.Date  Net.Due.Dt  \
0   Not OD            0.10  ...  31.12.2018   Dec   31-12-18    05-01-19   
1  Overdue            0.04  ...  31.10.2018   Oct   31-10-18    10-11-18   
2   Not OD            0.12  ...  05.11.2018   Nov   05-11-18    15-11-18   
3  Overdue            0.17  ...  30.12.2018   Dec   30-12-18    09-01-19   
4  Overdue            0.16  ...  03.11.2018   Nov   03-11-18    13-11-18   

       G.L  Year Clrng.doc.  Cl

In [6]:
df4.drop(columns=['Status','Clrng.doc.'],inplace=True)

In [8]:
import time
t1=time.time()
print('Training Starting............')
clf=trainer(df2,y)
print('Training Done...................')
print('Time Taken',time.time()-t1)
print('-----------------------------------------------------------------------------------')

Training Starting............
Train on 73933 samples, validate on 31686 samples
Epoch 1/20
73933/73933 [==============================] - 21s 278us/step - loss: 4.2119 - acc: 0.7387 - val_loss: 4.2114 - val_acc: 0.7387
Epoch 2/20
73933/73933 [==============================] - 16s 222us/step - loss: 4.2119 - acc: 0.7387 - val_loss: 4.2114 - val_acc: 0.7387
Epoch 3/20
73933/73933 [==============================] - 22s 299us/step - loss: 4.2119 - acc: 0.7387 - val_loss: 4.2114 - val_acc: 0.7387
Epoch 4/20
73933/73933 [==============================] - 15s 205us/step - loss: 4.2119 - acc: 0.7387 - val_loss: 4.2114 - val_acc: 0.7387
Epoch 5/20
73933/73933 [==============================] - 15s 198us/step - loss: 4.2119 - acc: 0.7387 - val_loss: 4.2114 - val_acc: 0.7387
Epoch 6/20
73933/73933 [==============================] - 16s 211us/step - loss: 4.2119 - acc: 0.7387 - val_loss: 4.2114 - val_acc: 0.7387
Epoch 7/20
73933/73933 [==============================] - 16s 220us/step - loss: 4.211

In [7]:
print('Feature Importances')
for feature in zip(df2.columns, clf.feature_importances_):
    print(feature)

Feature Importances
('BusA', 0.05241632542538666)
('CCAr', 0.0)
('Account', 0.1560521962611053)
('Zone', 0.036105220587334855)
('Bran', 0.08643051659817977)
('Local.Crcy.Amt', 0.10206840844131058)
('PayT', 0.06511257942943238)
('Month', 0.06297018491231879)
('Pstng.Date', 0.09127714655149731)
('Net.Due.Dt', 0.18673246930283183)
('Pending_Amy', 0.08698595119260595)
('pending_cnt', 0.07384900129799658)


In [8]:
testpath= ''
#test=pd.read_csv(testpath)
predict(df4,clf,busa,ccar,month,zone,bran,payt)
print() 
print('______________________________________  END  ____________________________________')
print()
print()

(2959, 22)
(2959, 22)
Starting Prediction ....


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Done 300 out of 300 | elapsed:    0.4s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    0.0s


Generating Probabilities.....


[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done 300 out of 300 | elapsed:    0.4s finished


Finished prediction...

   BusA  CCAr  Account  Zone  Bran  Local.Crcy.Amt  PayT  Month  Pstng.Date  \
0     6     0   927881     2    21            0.17     0      2    20180712   
1     6     0   927881     2    21            0.16     0      3    20180612   
2     6     0   927881     2    21            0.13     0      3    20180612   
3     6     0   927881     2    21            0.17     0      6    20181012   
4     0     0   927881     2    21            0.14     0      6    20181012   

   Net.Due.Dt  Pending_Amy  pending_cnt  
0    20180712          0.0            0  
1    20180612          0.0            0  
2    20180612          0.0            0  
3    20181012          0.0            0  
4    20181012          0.0            0  
Computing Prediction Table............
   BusA CCAr     Reference  Account         Customer.Name   Zone  Bran  Ty  \
0  9600  BPR  4.130033e+09   927881  SAMRAT IRONS PVT LTD  South  SBAD  RV   
1  9600  BPR  4.130032e+09   927881  SAMRAT IRONS PVT 